# Load Data

# Install Library yang dibutuhkan

In [1]:
import moviepy
print(moviepy.__file__)

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\moviepy\__init__.py


In [2]:
from __future__ import division
import matplotlib.pyplot as plt
import cv2
import os, glob
import numpy as np
from moviepy.editor import VideoFileClip
cwd = os.getcwd()



ModuleNotFoundError: No module named 'moviepy.editor'

In [2]:
import sys
print(sys.executable)


C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


# fungsi show_images:tentukan tata letak, buat figure, tampilkan setiap gambar

In [ ]:
def show_images(images, cmap=None):
    cols = 2 # tentukan tata letak
    rows = (len(images)+1)//cols # jumlah baris dihitung berdasarkan jumlah gambar dalam kolom

    plt.figure(figsize=(15, 12)) # membuat figure baru
    for i, image in enumerate(images): # mengiterasi setiap gambar dalam daftar
        plt.subplot(rows, cols, i+1) # membuat subplot untuk setiap gambar
        # Gunakan peta warna skala abu-abu jika hanya ada satu saluran
        cmap = 'gray' if len(image.shape)==2 else cmap
        plt.imshow(image, cmap=cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

In [ ]:
test_images = [plt.imread(path) for path in glob.glob('/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/test_images/*.jpg')]

show_images(test_images)

<Figure size 1500x1200 with 0 Axes>

# Prepocessing

## Seleksi Warna (Color Selection)
# membuat filter warna
# menggabungkan filter
# menerapkan filter
# mengembalikan gambar


In [ ]:
# Dua masker warna dibuat, satu untuk putih dan satu untuk kuning, menggunakan rentang nilai RGB tertentu.
# cv2.inRange digunakan untuk membuat masker, piksel dalam rentang warna akan bernilai 1 (putih) dan di luar rentang akan bernilai 0 (hitam).

# image is expected be in RGB color space# image
def select_rgb_white_yellow(image):
    # filter warna putih
    lower = np.uint8([120, 120, 120])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(image, lower, upper)
    # filter warna kuning
    lower = np.uint8([190, 190,   0])
    upper = np.uint8([255, 255, 255])
    yellow_mask = cv2.inRange(image, lower, upper)
    # menggabungkan filter
    # Filter putih dan kuning digabungkan menggunakan cv2.bitwise_or. Hasilnya, piksel yang merupakan warna putih atau kuning akan bernilai 1
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    # Masker yang digabungkan diterapkan pada gambar asli
    # Hanya piksel yang bernilai 1 pada filter (warna putih atau kuning) yang akan dipertahankan pada gambar hasil.
    masked = cv2.bitwise_and(image, image, mask = mask)
    return masked

#Fungsi mengembalikan gambar yang telah difilter, hanya menampilkan area berwarna putih dan kuning
white_yellow_images = list(map(select_rgb_white_yellow, test_images))
show_images(white_yellow_images)

<Figure size 1500x1200 with 0 Axes>

# Konversi ke Skala Abu-abu (Grayscale Conversion)

In [ ]:
def convert_gray_scale(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # konversi warna
    # flag yang menunjukkan konversi dari ruang warna RGB (Red, Green, Blue) ke grayscale
    # Fungsi ini mengembalikan gambar yang telah dikonversi ke grayscale.

#Hasilnya disimpan dalam list baru bernama gray_images, yang berisi gambar-gambar dalam format grayscale.
gray_images = list(map(convert_gray_scale, white_yellow_images))

# menampilkan gambar-gambar dalam list
show_images(gray_images)

<Figure size 1500x1200 with 0 Axes>

#  Deteksi Tepi (Edge Detection) Canny

In [ ]:
# OpenCV digunakan untuk mendeteksi tepi pada gambar menggunakan algoritma Canny
def detect_edges(image, low_threshold=50, high_threshold=200):
    return cv2.Canny(image, low_threshold, high_threshold)

edge_images = list(map(lambda image: detect_edges(image), gray_images))

show_images(edge_images)

<Figure size 1500x1200 with 0 Axes>

### identifikasi  Pemilihan Area (Region of Interest)

In [ ]:
def filter_region(image, vertices):
    """
    Create the mask using the vertices and apply it to the input image
    """
    mask = np.zeros_like(image)
    if len(mask.shape)==2:
        cv2.fillPoly(mask, vertices, 255)
    else:
        cv2.fillPoly(mask, vertices, (255,)*mask.shape[2]) # in case, the input image has a channel dimension
    return cv2.bitwise_and(image, mask)


def select_region(image):
    """
    It keeps the region surrounded by the `vertices` (i.e. polygon).  Other area is set to 0 (black).
    """
    # first, define the polygon by vertices
    rows, cols = image.shape[:2]
    pt_1  = [cols*0.05, rows*0.90]
    pt_2 = [cols*0.05, rows*0.70]
    pt_3 = [cols*0.30, rows*0.55]
    pt_4 = [cols*0.6, rows*0.15]
    pt_5 = [cols*0.90, rows*0.15]
    pt_6 = [cols*0.90, rows*0.90]
    # the vertices are an array of polygons (i.e array of arrays) and the data type must be integer
    vertices = np.array([[pt_1, pt_2, pt_3, pt_4, pt_5, pt_6]], dtype=np.int32)
    return filter_region(image, vertices)


# images showing the region of interest only
roi_images = list(map(select_region, edge_images))

show_images(roi_images)

<Figure size 1500x1200 with 0 Axes>

# Ekstrasi Fitur

###  Transformasi Hough Line (Hough Line Transform

In [ ]:
def hough_lines(image):
    """
    `image` should be the output of a Canny transform.

    Returns hough lines (  the image with lines)
    """
    return cv2.HoughLinesP(image, rho=0.1, theta=np.pi/10, threshold=15, minLineLength=9, maxLineGap=4)


list_of_lines = list(map(hough_lines, roi_images))

# menggambar garis-garis yang terdeteksi oleh Hough Line Transform pada citra asli

In [ ]:
def draw_lines(image, lines, color=[255, 0, 0], thickness=2, make_copy=True):
    # the lines returned by cv2.HoughLinesP has the shape (-1, 1, 4)
    if make_copy:
        image = np.copy(image) # don't want to modify the original
    cleaned = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if abs(y2-y1) <=1 and abs(x2-x1) >=25 and abs(x2-x1) <= 55:
                cleaned.append((x1,y1,x2,y2))
                cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    print(" No lines detected: ", len(cleaned))
    return image


line_images = []
for image, lines in zip(test_images, list_of_lines):
    line_images.append(draw_lines(image, lines))

show_images(line_images)

<Figure size 1500x1200 with 0 Axes>

## Identifikasi blok parkir persegi panjang

In [ ]:
def identify_blocks(image, lines, make_copy=True):
    if make_copy:
        new_image = np.copy(image)
    #Step 1: Create a clean list of lines
    cleaned = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if abs(y2-y1) <=1 and abs(x2-x1) >=25 and abs(x2-x1) <= 55:
                cleaned.append((x1,y1,x2,y2))

    #Step 2: Sort cleaned by x1 position
    import operator
    list1 = sorted(cleaned, key=operator.itemgetter(0, 1))

    #Step 3: Find clusters of x1 close together - clust_dist apart
    clusters = {}
    dIndex = 0
    clus_dist = 10

    for i in range(len(list1) - 1):
        distance = abs(list1[i+1][0] - list1[i][0])
    #         print(distance)
        if distance <= clus_dist:
            if not dIndex in clusters.keys(): clusters[dIndex] = []
            clusters[dIndex].append(list1[i])
            clusters[dIndex].append(list1[i + 1])

        else:
            dIndex += 1

    #Step 4: Identify coordinates of rectangle around this cluster
    rects = {}
    i = 0
    for key in clusters:
        all_list = clusters[key]
        cleaned = list(set(all_list))
        if len(cleaned) > 5:
            cleaned = sorted(cleaned, key=lambda tup: tup[1])
            avg_y1 = cleaned[0][1]
            avg_y2 = cleaned[-1][1]
    #         print(avg_y1, avg_y2)
            avg_x1 = 0
            avg_x2 = 0
            for tup in cleaned:
                avg_x1 += tup[0]
                avg_x2 += tup[2]
            avg_x1 = avg_x1/len(cleaned)
            avg_x2 = avg_x2/len(cleaned)
            rects[i] = (avg_x1, avg_y1, avg_x2, avg_y2)
            i += 1

    print("Num Parking Lanes: ", len(rects))
    #Step 5: Draw the rectangles on the image
    buff = 7
    for key in rects:
        tup_topLeft = (int(rects[key][0] - buff), int(rects[key][1]))
        tup_botRight = (int(rects[key][2] + buff), int(rects[key][3]))
#         print(tup_topLeft, tup_botRight)
        cv2.rectangle(new_image, tup_topLeft,tup_botRight,(0,255,0),3)
    return new_image, rects

# images showing the region of interest only
rect_images = []
rect_coords = []
for image, lines in zip(test_images, list_of_lines):
    new_image, rects = identify_blocks(image, lines)
    rect_images.append(new_image)
    rect_coords.append(rects)

show_images(rect_images)

<Figure size 1500x1200 with 0 Axes>

## Identifikasi setiap tempat dan hitung jumlah tempat parkir

Langkah selanjutnya-

Berdasarkan lebar setiap segmen garis parkir ke tempat individu,
Gambar visualisasi semua tempat parkir

In [ ]:
def draw_parking(image, rects, make_copy = True, color=[255, 0, 0], thickness=2, save = True):
    if make_copy:
        new_image = np.copy(image)
    gap = 15.5
    spot_dict = {} # maps each parking ID to its coords
    tot_spots = 0
    adj_y1 = {0: 20, 1:-10, 2:0, 3:-11, 4:28, 5:5, 6:-15, 7:-15, 8:-10, 9:-30, 10:9, 11:-32}
    adj_y2 = {0: 30, 1: 50, 2:15, 3:10, 4:-15, 5:15, 6:15, 7:-20, 8:15, 9:15, 10:0, 11:30}

    adj_x1 = {0: -8, 1:-15, 2:-15, 3:-15, 4:-15, 5:-15, 6:-15, 7:-15, 8:-10, 9:-10, 10:-10, 11:0}
    adj_x2 = {0: 0, 1: 15, 2:15, 3:15, 4:15, 5:15, 6:15, 7:15, 8:10, 9:10, 10:10, 11:0}
    for key in rects:
        # Horizontal lines
        tup = rects[key]
        x1 = int(tup[0]+ adj_x1[key])
        x2 = int(tup[2]+ adj_x2[key])
        y1 = int(tup[1] + adj_y1[key])
        y2 = int(tup[3] + adj_y2[key])
        cv2.rectangle(new_image, (x1, y1),(x2,y2),(0,255,0),2)
        num_splits = int(abs(y2-y1)//gap)
        for i in range(0, num_splits+1):
            y = int(y1 + i*gap)
            cv2.line(new_image, (x1, y), (x2, y), color, thickness)
        if key > 0 and key < len(rects) -1 :
            #draw vertical lines
            x = int((x1 + x2)/2)
            cv2.line(new_image, (x, y1), (x, y2), color, thickness)
        # Add up spots in this lane
        if key == 0 or key == (len(rects) -1):
            tot_spots += num_splits +1
        else:
            tot_spots += 2*(num_splits +1)

        # Dictionary of spot positions
        if key == 0 or key == (len(rects) -1):
            for i in range(0, num_splits+1):
                cur_len = len(spot_dict)
                y = int(y1 + i*gap)
                spot_dict[(x1, y, x2, y+gap)] = cur_len +1
        else:
            for i in range(0, num_splits+1):
                cur_len = len(spot_dict)
                y = int(y1 + i*gap)
                x = int((x1 + x2)/2)
                spot_dict[(x1, y, x, y+gap)] = cur_len +1
                spot_dict[(x, y, x2, y+gap)] = cur_len +2

    print("total parking spaces: ", tot_spots, cur_len)
    if save:
        filename = 'with_parking.jpg'
        cv2.imwrite(filename, new_image)
    return new_image, spot_dict

delineated = []
spot_pos = []
for image, rects in zip(test_images, rect_coords):
    new_image, spot_dict = draw_parking(image, rects)
    delineated.append(new_image)
    spot_pos.append(spot_dict)

show_images(delineated)

<Figure size 1500x1200 with 0 Axes>

In [ ]:
final_spot_dict = spot_pos[1]

IndexError: list index out of range

In [ ]:
print(len(final_spot_dict))

In [ ]:
def assign_spots_map(image, spot_dict=final_spot_dict, make_copy = True, color=[255, 0, 0], thickness=2):
    if make_copy:
        new_image = np.copy(image)
    for spot in spot_dict.keys():
        (x1, y1, x2, y2) = spot
        cv2.rectangle(new_image, (int(x1),int(y1)), (int(x2),int(y2)), color, thickness)
    return new_image

marked_spot_images = list(map(assign_spots_map, test_images))
show_images(marked_spot_images)

In [ ]:
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path lengkap ke file di Google Drive
file_path = '/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/spot_dict.pickle'

# Simpan dictionary ke file pickle
with open(file_path, 'wb') as handle:
    pickle.dump(final_spot_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
### Save spot dictionary as pickle file
import pickle

with open('spot_dict.pickle', 'wb') as handle:
    pickle.dump(final_spot_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Save image for CNN model

In [ ]:
import os

   # Path ke folder di Google Drive
folder_path = '/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/for_cnn'  # Ganti 'MyDrive' dengan folder tujuan di Drive kamu

   # Buat folder jika belum ada
if not os.path.exists(folder_path):
       os.makedirs(folder_path)

In [ ]:
def save_images_for_cnn(image, spot_dict=final_spot_dict, folder_name='for_cnn'):
    for spot in spot_dict.keys():
        (x1, y1, x2, y2) = spot
        (x1, y1, x2, y2) = (int(x1), int(y1), int(x2), int(y2))
        # crop this image
        spot_img = image[y1:y2, x1:x2]
        spot_img = cv2.resize(spot_img, (0, 0), fx=2.0, fy=2.0)
        spot_id = spot_dict[spot]

        filename = 'spot' + str(spot_id) + '.jpg'
        print(spot_img.shape, filename, (x1, x2, y1, y2))

        # Simpan gambar ke folder 'for_cnn'
        cv2.imwrite(os.path.join(folder_name, filename), spot_img)

In [ ]:
# ... (kode sebelumnya)
marked_spot_images = list(map(assign_spots_map, test_images))
show_images(marked_spot_images)

# Panggil fungsi save_images_for_cnn di sini
for image in test_images:
    save_images_for_cnn(image)

In [ ]:
import os

# Cetak path lengkap ke folder
print(os.path.abspath('for_cnn'))

## mulai


In [ ]:
!pip install tensorflow

In [ ]:
import numpy
import os
# Import from tensorflow.keras instead of keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [ ]:
files_train = 0
files_validation = 0

cwd = os.getcwd()
folder = '/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/train'
for sub_folder in os.listdir(folder):
    path, dirs, files = next(os.walk(os.path.join(folder,sub_folder)))
    files_train += len(files)


folder = '/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/test'
for sub_folder in os.listdir(folder):
    path, dirs, files = next(os.walk(os.path.join(folder,sub_folder)))
    files_validation += len(files)

print(files_train,files_validation)

# tetapkan parameter utama

In [ ]:
img_width, img_height = 48, 48
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/train"
validation_data_dir = "/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/test"
nb_train_samples = files_train
nb_validation_samples = files_validation
batch_size = 32
epochs = 15
num_classes = 2

# Bangun model di atas VGG terlatih

In [ ]:
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:10]:
    layer.trainable = False

In [ ]:
# ... (kode sebelumnya di sel ipython-input-53-8a42504ea456)

# Tambahkan baris ini setelah memuat model VGG16
x = model.output
x = Flatten()(x)  # Ratakan output
x = Dense(num_classes, activation='softmax')(x)  # Tambahkan layer Dense dengan aktivasi softmax

# Buat model baru
model = Model(inputs=model.input, outputs=x)

# Kompilasi model setelah modifikasi  <-- Baris ini yang ditambahkan
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.SGD(learning_rate=0.0001, momentum=0.9),
              metrics=["accuracy"])

# ... (sisa kode)

In [ ]:
# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.1,
width_shift_range = 0.1,
height_shift_range=0.1,
rotation_range=5)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.1,
width_shift_range = 0.1,
height_shift_range=0.1,
rotation_range=5)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size,
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

In [ ]:
# Save the model according to the conditions
checkpoint = ModelCheckpoint("car1.keras", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)  # Change 'period' to 'save_freq' and filepath to 'car1.keras'
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='max') # Add mode='max' to EarlyStopping

In [ ]:
### Start training!
# The variable 'model_final' was not defined. Using 'model' instead, assuming it contains the compiled model
history_object = model.fit( # Changed fit_generator to fit, as fit_generator is deprecated
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size, # Added steps_per_epoch, required for fit
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size, # Added validation_steps, required for fit
    callbacks = [checkpoint, early]
)

In [ ]:
import matplotlib.pyplot as plt

print(history_object.history.keys())

# Plot akurasi
plt.plot(history_object.history['accuracy'])
plt.plot(history_object.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Tampilkan akurasi detail
train_accuracy = history_object.history['accuracy'][-1]
val_accuracy = history_object.history['val_accuracy'][-1]

print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Validation Accuracy: {val_accuracy:.2%}")

In [ ]:
import matplotlib.pyplot as plt

print(history_object.history.keys())

# Plot akurasi
plt.plot(history_object.history['accuracy'])
plt.plot(history_object.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')

# Tampilkan skor training dan validation di gambar
train_accuracy = history_object.history['accuracy'][-1]
val_accuracy = history_object.history['val_accuracy'][-1]
plt.text(len(history_object.history['accuracy']) - 1, train_accuracy, f'Train: {train_accuracy:.2%}', ha='right', va='bottom')
plt.text(len(history_object.history['val_accuracy']) - 1, val_accuracy, f'Val: {val_accuracy:.2%}', ha='right', va='bottom')


plt.show()

# Tampilkan akurasi detail (opsional, bisa dihapus jika sudah ditampilkan di gambar)
# print(f"Training Accuracy: {train_accuracy:.2%}")
# print(f"Validation Accuracy: {val_accuracy:.2%}")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Tampilkan loss detail
train_loss = history_object.history['loss'][-1]
val_loss = history_object.history['val_loss'][-1]

print(f"Training Loss: {train_loss:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

# Tampilkan loss detail di dalam gambar
train_loss = history_object.history['loss'][-1]
val_loss = history_object.history['val_loss'][-1]
plt.text(len(history_object.history['loss']) - 1, train_loss, f'Train: {train_loss:.4f}', ha='right', va='bottom')
plt.text(len(history_object.history['val_loss']) - 1, val_loss, f'Val: {val_loss:.4f}', ha='right', va='bottom')

plt.show()

# Tampilkan loss detail (opsional, bisa dihapus jika sudah ditampilkan di gambar)
#print(f"Training Loss: {train_loss:.4f}")
#print(f"Validation Loss: {val_loss:.4f}")

In [ ]:
checkpoint = ModelCheckpoint("car1.keras", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)

In [ ]:
from google.colab import files
import os

# Get the absolute path of the file
file_path = os.path.abspath('car1.keras')  # Assuming car1.keras is in the current directory

# Check if the file exists
if os.path.exists(file_path):
  files.download(file_path)
else:
  print(f"Error: File not found at {file_path}")

In [ ]:
import os
print(os.getcwd()) # Menampilkan direktori kerja saat ini
print(os.listdir()) # Menampilkan isi direktori kerja

In [ ]:
model.save('car1.keras')

## Evaluasi Model

In [ ]:
# Import library yang dibutuhkan
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.models import load_model

# Load model yang sudah dilatih
model = load_model('car1.keras') # Ganti dengan path model Anda

# Load data test
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Jangan acak data untuk evaluasi
)

# Prediksi pada data test
Y_pred = model.predict(validation_generator, nb_validation_samples // batch_size + 1)  # Prediksi menggunakan model
y_pred = np.argmax(Y_pred, axis=1)  # Dapatkan kelas yang diprediksi
y_true = validation_generator.classes  # Dapatkan label sebenarnya


# Evaluasi model
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))  # Menampilkan confusion matrix

print('Classification Report')
target_names = ['empty', 'occupied']  # Nama kelas
print(classification_report(y_true, y_pred, target_names=target_names))  # Menampilkan laporan klasifikasi

# Hitung metrik evaluasi
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
accuracy = accuracy_score(y_true, y_pred)
print(f"Akurasi: {accuracy:.2%}")

In [ ]:
import numpy as np
import pandas as pd

# Membuat dan menampilkan confusion matrix dalam bentuk tabel
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=['empty', 'occupied'], columns=['empty', 'occupied'])

print("Confusion Matrix:")
print(cm_df)

In [ ]:
# Membuat dan menampilkan confusion matrix dalam bentuk gambar
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['empty', 'occupied'],
            yticklabels=['empty', 'occupied'])
plt.title('Confusion Matrix')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
## Imports for making predictions
from PIL import Image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
from keras.preprocessing import image

In [ ]:
cwd = os.getcwd()
top_model_weights_path = '/content/car1.keras'

class_dictionary = {}
class_dictionary[0] = 'empty'
class_dictionary[1] = 'occupied'

In [ ]:
from PIL import Image
model = load_model(top_model_weights_path)

In [ ]:
def make_prediction(image):
    #Rescale image
    img = image/255.

    #Convert to a 4D tensor
    image = np.expand_dims(img, axis=0)
    #print(image.shape)

    # make predictions on the preloaded model
    class_predicted = model.predict(image)
    inID = np.argmax(class_predicted[0])
    label = class_dictionary[inID]
    return label

In [ ]:
def predict_on_image(image, spot_dict = final_spot_dict, make_copy=True, color = [0, 255, 0], alpha=0.5):
    if make_copy:
        new_image = np.copy(image)
        overlay = np.copy(image)
    cnt_empty = 0
    all_spots = 0
    for spot in spot_dict.keys():
        all_spots += 1
        (x1, y1, x2, y2) = spot
        (x1, y1, x2, y2) = (int(x1), int(y1), int(x2), int(y2))
        #crop this image
        spot_img = image[y1:y2, x1:x2]
        spot_img = cv2.resize(spot_img, (48, 48))

        label = make_prediction(spot_img)
#         print(label)
        if label == 'empty':
            cv2.rectangle(overlay, (int(x1),int(y1)), (int(x2),int(y2)), color, -1)
            cnt_empty += 1

    cv2.addWeighted(overlay, alpha, new_image, 1 - alpha, 0, new_image)

    cv2.putText(new_image, "Available: %d spots" %cnt_empty, (30, 95),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (255, 255, 255), 2)

    cv2.putText(new_image, "Total: %d spots" %all_spots, (30, 125),
    cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (255, 255, 255), 2)
    save = False

    if save:
        filename = 'with_marking.jpg'
        cv2.imwrite(filename, new_image)

    return new_image


predicted_images = list(map(predict_on_image, test_images))
show_images(predicted_images)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# Hitung jumlah data latih dan data uji
files_train = 0
files_validation = 0

folder_train = "/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/train"
for sub_folder in os.listdir(folder_train):
    path, dirs, files = next(os.walk(os.path.join(folder_train, sub_folder)))
    files_train += len(files)

folder_test = "/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/test"
for sub_folder in os.listdir(folder_test):
    path, dirs, files = next(os.walk(os.path.join(folder_test, sub_folder)))
    files_validation += len(files)

# Data untuk grafik
categories = ['Train Data', 'Test Data']
values = [files_train, files_validation]

# Membuat grafik batang
plt.figure(figsize=(8, 6))
plt.bar(categories, values, color=['blue', 'orange'])
plt.title('Jumlah Data Latih dan Data Uji')
plt.xlabel('Kategori Data')
plt.ylabel('Jumlah Data')

# Menambahkan label nilai di atas setiap batang
for i, v in enumerate(values):
    plt.text(i, v + 5, str(v), ha='center', va='bottom')

plt.show()

In [ ]:
import os

def hitung_total_data(folder_path):
    total_count = 0
    for sub_folder in os.listdir(folder_path):
        path, dirs, files = next(os.walk(os.path.join(folder_path, sub_folder)))
        total_count += len(files)
    return total_count

train_data_dir = "/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/train"
validation_data_dir = "/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/test"

total_train_data = hitung_total_data(train_data_dir)
total_test_data = hitung_total_data(validation_data_dir)

total_data = total_train_data + total_test_data

print("Total Data Latih:", total_train_data)
print("Total Data Uji:", total_test_data)
print("Total Data:", total_data)

In [ ]:
import os

def hitung_distribusi_kelas(folder_path):
    empty_count = len(os.listdir(os.path.join(folder_path, 'empty')))
    occupied_count = len(os.listdir(os.path.join(folder_path, 'occupied')))
    return empty_count, occupied_count

train_empty_count, train_occupied_count = hitung_distribusi_kelas(train_data_dir)
test_empty_count, test_occupied_count = hitung_distribusi_kelas(validation_data_dir)

print("Data Latih:")
print("Kosong:", train_empty_count)
print("Terisi:", train_occupied_count)

print("\nData Uji:")
print("Kosong:", test_empty_count)
print("Terisi:", test_occupied_count)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data distribusi kelas (dari kode Anda sebelumnya)
train_empty_count, train_occupied_count = hitung_distribusi_kelas(train_data_dir)
test_empty_count, test_occupied_count = hitung_distribusi_kelas(validation_data_dir)

# Data untuk grafik
categories = ['Kosong', 'Terisi']
train_values = [train_empty_count, train_occupied_count]
test_values = [test_empty_count, test_occupied_count]

# Membuat grafik batang dengan ukuran kecil
fig, ax = plt.subplots(figsize=(4, 3))  # Mengatur ukuran grafik (lebar, tinggi)

bar_width = 0.35  # Lebar setiap batang
index = np.arange(len(categories))

rects1 = ax.bar(index, train_values, bar_width, label='Data Latih', color='blue')
rects2 = ax.bar(index + bar_width, test_values, bar_width, label='Data Uji', color='orange')

# Menambahkan label, judul, dan legenda
ax.set_xlabel('Kelas')
ax.set_ylabel('Jumlah Gambar')
ax.set_title('Distribusi Kelas Data Latih dan Data Uji', fontsize=10)  # Mengatur ukuran font judul
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(categories)
ax.legend(fontsize=8)  # Mengatur ukuran font legenda

# Menambahkan label nilai di atas setiap batang
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2, height,
                '%d' % int(height), ha='center', va='bottom', fontsize=8)  # Mengatur ukuran font label nilai

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

# Path ke folder data latih
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/train_data/train"

# Mendapatkan path ke gambar contoh
empty_image_path = os.path.join(train_data_dir, 'empty', os.listdir(os.path.join(train_data_dir, 'empty'))[0])  # Path gambar kosong pertama
occupied_image_path = os.path.join(train_data_dir, 'occupied', os.listdir(os.path.join(train_data_dir, 'occupied'))[0])  # Path gambar terisi pertama


# Membaca dan menampilkan gambar
img_empty = cv2.imread(empty_image_path)
img_occupied = cv2.imread(occupied_image_path)

plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img_empty, cv2.COLOR_BGR2RGB))
plt.title('Contoh Tempat Parkir Kosong')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(img_occupied, cv2.COLOR_BGR2RGB))
plt.title('Contoh Tempat Parkir Terisi')
plt.axis('off')

plt.tight_layout()
plt.show()

### Run code on video

In [ ]:
pip install streamlit opencv-python pillow

In [ ]:
!pip install opencv-python-headless

In [ ]:
!pip install opencv-python-headless

In [ ]:
import cv2

# Path ke file video
video_path = "/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/parking_video.mp4"

# Membuka video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Tidak dapat membuka video.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Contoh proses frame: ubah ke grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Print ukuran frame (sebagai contoh, tidak ditampilkan langsung)
        print(f"Frame size: {gray_frame.shape}")

cap.release()


In [ ]:
# Tentukan codec dan buat VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 30.0, (640, 480))

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Proses frame jika diperlukan (misalnya, tambahkan filter atau deteksi objek)
    out.write(frame)

cap.release()
out.release()


In [ ]:
from IPython.display import HTML
from IPython.display import Video

# Tampilkan video dalam format HTML
HTML("""
<video width=600 controls>
    <source src="output_video.mp4" type="video/mp4">
</video>
""")


In [ ]:
Video('output_video.mp4', embed=True)

In [ ]:
from google.colab.patches import cv2_imshow
video_name = '/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/parking_video.mp4'
cap = cv2.VideoCapture(video_name)
ret = True
count = 0

while ret:
        ret, image = cap.read()
        count += 1
        if count == 5:
            count = 0

            new_image = np.copy(image)
            overlay = np.copy(image)
            cnt_empty = 0
            all_spots = 0
            color = [0, 255, 0]
            alpha=0.5
            for spot in final_spot_dict.keys():
                all_spots += 1
                (x1, y1, x2, y2) = spot
                (x1, y1, x2, y2) = (int(x1), int(y1), int(x2), int(y2))
                #crop this image
                spot_img = image[y1:y2, x1:x2]
                spot_img = cv2.resize(spot_img, (48, 48))

                label = make_prediction(spot_img)
        #         print(label)
                if label == 'empty':
                    cv2.rectangle(overlay, (int(x1),int(y1)), (int(x2),int(y2)), color, -1)
                    cnt_empty += 1

            cv2.addWeighted(overlay, alpha, new_image, 1 - alpha, 0, new_image)

            cv2.putText(new_image, "Available: %d spots" %cnt_empty, (30, 95),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7, (255, 255, 255), 2)

            cv2.putText(new_image, "Total: %d spots" %all_spots, (30, 125),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7, (255, 255, 255), 2)
            cv2.imshow('frame', new_image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        #out.write(image)

cv2.destroyAllWindows()
cap.release()

In [ ]:
from IPython.display import display, clear_output
from PIL import Image

video_name = '/content/drive/MyDrive/Colab Notebooks/parking_spots_detector/parking_video.mp4'
cap = cv2.VideoCapture(video_name)
ret = True

# FPS adjustment
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps)

while ret:
    ret, frame = cap.read()
    if ret:
        # Convert frame to RGB for PIL.Image
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame_rgb)

        # Prediksi dan gambar kotak
        predicted_frame = predict_on_image(frame, final_spot_dict, make_copy=True)
        predicted_rgb = cv2.cvtColor(predicted_frame, cv2.COLOR_BGR2RGB)
        predicted_image = Image.fromarray(predicted_rgb)

        # Tampilkan frame
        clear_output(wait=True)
        display(predicted_image)

        # Tunggu beberapa saat sebelum menampilkan frame berikutnya
        # cv2.waitKey(delay)  # Diperlukan jika menggunakan cv2.imshow()
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()
cap.release()